<a href="https://colab.research.google.com/github/choycoy/final_project/blob/main/prediction_snn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")
q_df['question_id'] = q_df['question_id'].str.replace('q', '')
q_df

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,1,b1,e1,b,1,1;2;179;181,1558093217098
1,2,b2,e2,a,1,15;2;182,1558093219720
2,3,b3,e3,b,1,14;2;179;183,1558093222784
3,4,b4,e4,b,1,9;2;179;184,1558093225357
4,5,b5,e5,c,1,8;2;179;181,1558093228439
...,...,...,...,...,...,...,...
13164,18139,b12202,e12202,b,2,24;26;183;182,1571733814684
13165,18140,b12203,e12203,a,2,24;33;183;182,1571733815331
13166,18141,b12204,e12204,a,2,24;26;183;182,1571733815951
13167,18142,b12205,e12205,a,2,24;26;183;182,1571733816585


In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10])

In [ ]:
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)

             timestamp  solving_id  question_id user_answer  elapsed_time  \
0        1565096190868           1         5012           b         38000   
1        1565096221062           2         4706           c         24000   
2        1565096293432           3         4366           b         68000   
3        1565096339668           4         4829           a         42000   
4        1565096401774           5         6528           b         59000   
...                ...         ...          ...         ...           ...   
9999995  1496051910897          93         6243           d          8000   
9999996  1496051940123          94         6552           b         28000   
9999997  1496051951404          95         6442           d         10000   
9999998  1496051971598          96         5470           b         19000   
9999999  1496052005208          97         5198           d         32000   

         user_id correct_answer  is_correct  
0              1             

In [ ]:
# Assuming you have the dataframe df containing student interactions with columns: timestamp, solving_id, question_id, user_answer, elapsed_time, user_id, and is_correct (1 for correct, 0 for incorrect).

# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction

# Calculate average, minimum, and maximum elapsed time for each user
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')
merged_df['min_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('min')
merged_df['max_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('max')

# Drop unnecessary columns if needed
merged_df.drop(['correct_count', 'interaction_count'], axis=1, inplace=True)


0            0
1            1
2            2
3            2
4            2
          ... 
9999997    126
8999998    126
9999998    126
8999999    127
9999999    128
Name: correct_count, Length: 10000000, dtype: int64
0            1
1            2
2            3
3            4
4            5
          ... 
9999997    230
8999998    231
9999998    232
8999999    233
9999999    234
Name: interaction_count, Length: 10000000, dtype: int64


In [ ]:
merged_df['next_is_correct'] = merged_df['is_correct'].shift(-1)

In [ ]:
merged_df.dropna(subset=['next_is_correct'], inplace=True)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the dataframe merged_df with features and target variable 'next_is_correct'

# Split the data into features (X) and target variable (y)
features = ['correctness_rate', 'average_elapsed_time', 'min_elapsed_time', 'max_elapsed_time']
X = merged_df[features].values
y = merged_df['next_is_correct'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features (optional but can improve training)
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Initialize the neural network model
model = Sequential()

# Add layers to the model
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training data


In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=1024, validation_split=0.2)

# Make predictions on the testing data
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/50
6250/6250 [==============================] - 17s 2ms/step - loss: 0.6134 - accuracy: 0.6763 - val_loss: 0.6130 - val_accuracy: 0.6770
Epoch 2/50
6250/6250 [==============================] - 14s 2ms/step - loss: 0.6130 - accuracy: 0.6765 - val_loss: 0.6130 - val_accuracy: 0.6771
Epoch 3/50
6250/6250 [==============================] - 15s 2ms/step - loss: 0.6130 - accuracy: 0.6766 - val_loss: 0.6131 - val_accuracy: 0.6770
Epoch 4/50
6250/6250 [==============================] - 14s 2ms/step - loss: 0.6129 - accuracy: 0.6765 - val_loss: 0.6128 - val_accuracy: 0.6770
Epoch 5/50
6250/6250 [==============================] - 16s 3ms/step - loss: 0.6129 - accuracy: 0.6766 - val_loss: 0.6128 - val_accuracy: 0.6770
Epoch 6/50
6250/6250 [==============================] - 17s 3ms/step - loss: 0.6129 - accuracy: 0.6766 - val_loss: 0.6130 - val_accuracy: 0.6771
Epoch 7/50
6250/6250 [==============================] - 17s 3ms/step - loss: 0.6129 - accuracy: 0.6766 - val_loss: 0.6128 - val_ac